<a href="https://colab.research.google.com/github/hargurjeet/MachineLearning/blob/master/ML_using_Kfolds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dataset from Kaggle

In [ ]:
!pip install opendatasets --upgrade --quiet

In [75]:
import numpy as np
import pandas as pd
import opendatasets as od
from sklearn import model_selection

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [4]:
od.download('https://www.kaggle.com/c/30-days-of-ml/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hargurjeet
Your Kaggle Key: ··········


100%|██████████| 66.0M/66.0M [00:00<00:00, 127MB/s]



Extracting archive ./30-days-of-ml/30-days-of-ml.zip to ./30-days-of-ml


# Splitting the data to kfold

In [67]:
train_fold = pd.read_csv("/content/30-days-of-ml/train.csv")

test = pd.read_csv("/content/30-days-of-ml/test.csv", index_col=0)

In [68]:
train_fold["kfold"] = -1

In [69]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=df_train)):
    train_fold.loc[valid_indicies, "kfold"] = fold

In [70]:
train_fold.sample(5)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
35642,59201,B,A,A,C,B,D,A,E,G,G,0.361159,0.490055,0.295613,0.723724,0.276091,0.578631,0.542477,0.836905,1.030439,0.818731,0.816414,0.622834,0.644518,0.182101,7.424021,2
38411,63861,B,A,A,A,B,B,A,E,E,H,0.071099,0.623809,0.535494,0.588133,0.504585,0.049456,0.412099,0.385164,-0.259434,0.450364,0.299179,0.574148,0.279826,0.298761,8.162777,3
295631,492713,B,B,B,C,B,B,A,E,E,G,0.502966,0.743092,0.649385,0.460982,0.438885,0.407279,0.271530,0.241695,0.262866,0.351176,0.433664,0.291656,0.291326,0.290424,7.014565,1
118837,198036,B,A,A,C,B,C,A,E,E,F,0.732868,0.764295,0.364502,0.344101,0.516480,0.778639,0.688919,0.699218,0.610208,0.477269,0.111209,0.535028,0.756526,0.603765,7.443079,0
199624,332859,A,B,A,C,B,B,A,E,C,K,0.410964,0.364029,1.007522,0.442754,0.901304,0.392368,0.466577,0.374969,0.274610,0.181027,0.123459,0.072521,0.315281,0.530877,8.048620,2


In [71]:
train_fold.kfold.value_counts()

4    60000
3    60000
2    60000
1    60000
0    60000
Name: kfold, dtype: int64

In [72]:
train_fold.to_csv("train_folds.csv", index=False)

# Implementing Kfold on Random forests

In [73]:
train_fold.kfold

0         0
1         2
2         4
3         3
4         1
         ..
299995    4
299996    3
299997    1
299998    3
299999    1
Name: kfold, Length: 300000, dtype: int64

In [78]:
useful_features = [cname for cname in train_fold.columns if cname not in ['id','target','kfold' ]]
numeric_cols = [cname for cname in train_fold.columns if train_fold[cname].dtype in ['int64', 'float64']]
object_cols = [cname for cname in train_fold.columns if train_fold[cname].dtype in ['object']]

final_predictions = []
scores = []
for fold in range(5):
    xtrain = train_fold[train_fold.kfold != fold].reset_index(drop=True)
    xvalid = train_fold[train_fold.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    scaler = StandardScaler()
    xtrain[numeric_cols] = scaler.fit_transform(xtrain[numeric_cols])
    xvalid[numeric_cols] = scaler.transform(xvalid[numeric_cols])
    xtest[numeric_cols] = scaler.transform(xtest[numeric_cols])




    model = RandomForestRegressor(n_estimators=10, random_state=42)
    # model = XGBRegressor(random_state=fold)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

KeyError: ignored

In [79]:
numeric_cols = [cname for cname in train_fold.columns if train_fold[cname].dtype in ['int64', 'float64']]
xtrain[numeric_cols]

KeyError: ignored